Adaptado de:
https://github.com/piegu/language-models/blob/master/HuggingFace_Notebook_token_classification_NER_LeNER_Br.ipynb

# 1. Configs


In [15]:
#teste gpu
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb 20 21:37:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
#teste ram
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [17]:
#drive e variáveis globais
#utilizar GPU
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
_DATASET_TO_USE = 'pl_corpus' #lener_br

Mounted at /content/drive


In [18]:
#installs
%%capture
!pip install datasets transformers seqeval
!apt install git-lfs

In [19]:
#imports
import pathlib
from pathlib import Path
import pandas as pd
import transformers
import datasets
from datasets import Dataset, DatasetDict
from datasets import load_dataset, load_metric

print(transformers.__version__) 
print(datasets.__version__) 

4.16.2
1.18.3


In [20]:
#BERT pre-treinado Lener
model_checkpoint = "pierreguillou/bert-base-cased-pt-lenerbr"

#dtaset para fine-tunning
if _DATASET_TO_USE == 'lener_br':
  datasets = load_dataset("lener_br")
else:
  path_plcorpus_dataset = '/content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/pl_corpus_dataset/'  
  url_files = 'https://raw.githubusercontent.com/bergoliveira/disciplinaDL/main/pl_corpus/'
  datasets = load_dataset('/content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/pl_corpus_dataset/pl_corpus.py', 
                        data_files={'train': [url_files+'train.conll', url_files+'dev.conll'], 'test': url_files+'test.conll'})
#tarefa
task = "ner" 

#entidades no modelo IOB
#datasets["train"].features[f"ner_tags"]
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

Using custom data configuration default-7c613bab45c2950d
Reusing dataset pl_corpus (/root/.cache/huggingface/datasets/pl_corpus/default-7c613bab45c2950d/1.0.0/711af93da4644ff8dff30ef9308e16d45352ea7e9e58c9c40391900cfb3f06c5)


  0%|          | 0/3 [00:00<?, ?it/s]

['O',
 'B-ORGANIZACAO',
 'I-ORGANIZACAO',
 'B-PESSOA',
 'I-PESSOA',
 'B-DATA',
 'I-DATA',
 'B-LOCAL',
 'I-LOCAL',
 'B-FUNDAMENTO',
 'I-FUNDAMENTO',
 'B-PRODUTODELEI',
 'I-PRODUTODELEI',
 'B-EVENTO',
 'I-EVENTO']

In [21]:
#mostrar elementos do dataset
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,2452,[.],[O]
1,3818,[.],[O]
2,4144,[.],[O]
3,2894,[.],[O]
4,1443,[.],[O]
5,2480,"[Deputado, HEITOR, FREIRE, 2, Reportagem, veiculada, pelo, Valor, Econômico, .]","[B-PESSOA, B-PESSOA, I-PESSOA, O, O, O, O, B-LOCAL, I-LOCAL, O]"
6,1498,[.],[O]
7,3032,"[Nesse, sentido, ,, esperamos, que, a, adoção, das, medidas, propostas, dê, maior, liberdade, para, que, os, agentes, econômicos, fortalecem, seus, empreendimentos, ,, o, que, contribuiria, para, aumentar, os, investimentos, e, o, consumo, das, famílias, e, ,, consequentemente, ,, reduziria, os, níveis, de, desemprego, ,, razões, pelas, quais, contamos, com, o, apoio, dos, nobres, colegas, Parlamentares, para, o, aperfeiçoamento, e, a, aprovação, do, presente, Projeto, de, Lei, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PESSOA, O, O, O, O, O, O, O, O, O, O, O, O]"
8,285,"[Sala, das, Sessões, ,, em, de, março, de, 2011, .]","[O, O, O, O, O, O, B-DATA, I-DATA, I-DATA, O]"
9,1945,[.],[O]


# 2. Pre-processing

In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
label_all_tokens = True

#example
example = datasets["train"][5]
print(example["tokens"])

tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

['§', '1º', 'O', 'regulamento', 'disporá', 'sobre', 'a', 'definição', 'das', 'dimensões', 'dos', 'calçados', 'que', 'terão', 'sua', 'comercialização', 'vedada', ',', 'nos', 'termos', 'do', 'caput', 'deste', 'artigo']
['[CLS]', '§', '[UNK]', 'O', 'regulamento', 'disp', '##or', '##á', 'sobre', 'a', 'definição', 'das', 'dimensões', 'dos', 'cal', '##çados', 'que', 'terão', 'sua', 'comercialização', 've', '##dad', '##a', ',', 'nos', 'termos', 'do', 'cap', '##ut', 'deste', 'artigo', '[SEP]']
32 32


In [23]:
#tokenização
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenize_and_align_labels(datasets['train'][:5])
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/pl_corpus/default-7c613bab45c2950d/1.0.0/711af93da4644ff8dff30ef9308e16d45352ea7e9e58c9c40391900cfb3f06c5/cache-785385adef2b6b4c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/pl_corpus/default-7c613bab45c2950d/1.0.0/711af93da4644ff8dff30ef9308e16d45352ea7e9e58c9c40391900cfb3f06c5/cache-3a8ffc9f1bddbe0b.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/pl_corpus/default-7c613bab45c2950d/1.0.0/711af93da4644ff8dff30ef9308e16d45352ea7e9e58c9c40391900cfb3f06c5/cache-8550f6e31db87d82.arrow


# 3. Finetunning

In [24]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at pierreguillou/bert-base-cased-pt-lenerbr were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at pierreguillou/be

# 4. Treinamento

## 4.1 Connfigurações do modelo

In [25]:

model_name = model_checkpoint.split("/")[-1]

# hyperparameters utilizado no teste
per_device_batch_size = 5  #[5, 10, 20] cada batch por thread, duplicando os valores.
num_train_epochs = 55 # [35, 55, 75] pierre: we started with 10 epochs but the model overfits fastly
learning_rate = 2e-5 #[1e-4,2e-5,3e-4,5e-5] pierre: (AdamW) we started with 3e-4, then 1e-4, then 5e-5 but the model overfits fastly

# hyperparameters não modificados
gradient_accumulation_steps = 2
weight_decay = 0.01
save_total_limit = 100 ##early stopping
logging_steps = 290 #melhor evaluate frequently (5000 seems too high)
eval_steps = logging_steps
evaluation_strategy = 'epoch' #pierre; 'steps'
logging_strategy = 'epoch' #'pierre; 'steps'
save_strategy = 'epoch' #pierre; 'steps'
save_steps = logging_steps
load_best_model_at_end = True

fp16 = True

# folders
model_name = model_checkpoint.split("/")[-1]
folder_model = 'e' + str(num_train_epochs) + '_lr' + str(learning_rate)
if _DATASET_TO_USE == 'lener_br':
  output_dir = '/content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/output/' + 'ner-lenerbr-' + str(model_name) + '/checkpoints/' + folder_model
  logging_dir = '/content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/logs/' + 'ner-lenerbr-' + str(model_name) + '/logs/' + folder_model
else:
  output_dir = '/content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/output/' + 'ner-pl_corpus-' + str(model_name) + '/checkpoints/' + folder_model
  logging_dir = '/content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/logs/' + 'ner-pl_corpus-' + str(model_name) + '/logs/' + folder_model

# metrics
metric_for_best_model = 'eval_f1'
if metric_for_best_model == 'eval_f1':
    greater_is_better = True
elif metric_for_best_model == 'eval_loss':
    greater_is_better = False  

args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_batch_size, 
    per_device_eval_batch_size=per_device_batch_size*2,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs, 
    weight_decay=weight_decay,
    save_total_limit=save_total_limit,
    logging_steps = logging_steps,
    eval_steps = logging_steps,
    load_best_model_at_end = load_best_model_at_end,
    metric_for_best_model = metric_for_best_model,
    greater_is_better = greater_is_better,
    gradient_checkpointing = False,
    do_train = True,
    do_eval = True,
    do_predict = True,
    evaluation_strategy = evaluation_strategy,
    logging_dir=logging_dir, 
    logging_strategy = logging_strategy,
    save_strategy = save_strategy,
    save_steps = save_steps,
    fp16 = fp16,
    push_to_hub=False,
)

In [26]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")
#labels = [label_list[i] for i in example[f"{task}_tags"]]
#metric.compute(predictions=[labels], references=[labels])

In [27]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## 4.2 Execução

In [28]:
from transformers.trainer_callback import EarlyStoppingCallback

# wait early_stopping_patience x eval_steps before to stp the training in order to get a better model
early_stopping_patience = save_total_limit

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
)

trainer.train()

Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4770
  Num Epochs = 55
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 2
  Total optimization steps = 26235


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.217800,0.048687,0.669430,0.742529,0.704087,0.967723
2,0.046000,0.051966,0.770764,0.800000,0.785110,0.969318
3,0.021200,0.056218,0.725451,0.832184,0.775161,0.968544
4,0.010400,0.067275,0.774033,0.805172,0.789296,0.969177
5,0.010200,0.064943,0.773304,0.805747,0.789192,0.969248
6,0.007000,0.068317,0.718812,0.834483,0.772340,0.969576
7,0.005300,0.068885,0.772362,0.828736,0.799556,0.971054
8,0.004500,0.080689,0.757338,0.830460,0.792215,0.968661
9,0.005300,0.079672,0.754974,0.828736,0.790137,0.970233
10,0.003400,0.084221,0.763775,0.804598,0.783655,0.968403


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens.
***** Running Evaluation *****
  Num examples = 2479
  Batch size = 10
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/output/ner-pl_corpus-bert-base-cased-pt-lenerbr/checkpoints/e55_lr2e-05/checkpoint-477
Configuration saved in /content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/output/ner-pl_corpus-bert-base-cased-pt-lenerbr/checkpoints/e55_lr2e-05/checkpoint-477/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/output/ner-pl

TrainOutput(global_step=26235, training_loss=0.007261540871791574, metrics={'train_runtime': 4193.5626, 'train_samples_per_second': 62.56, 'train_steps_per_second': 6.256, 'total_flos': 7604718625743750.0, 'train_loss': 0.007261540871791574, 'epoch': 55.0})

# 5. Avaliação

In [29]:
#geral
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens.
***** Running Evaluation *****
  Num examples = 2479
  Batch size = 10


{'epoch': 55.0,
 'eval_accuracy': 0.971265041870939,
 'eval_f1': 0.8222285228239298,
 'eval_loss': 0.08889658749103546,
 'eval_precision': 0.8114157806379407,
 'eval_recall': 0.8333333333333334,
 'eval_runtime': 8.1595,
 'eval_samples_per_second': 303.816,
 'eval_steps_per_second': 30.394}

In [30]:
#predição por entidades
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens.
***** Running Prediction *****
  Num examples = 2479
  Batch size = 10


{'DATA': {'f1': 0.9047619047619048,
  'number': 157,
  'precision': 0.8491620111731844,
  'recall': 0.9681528662420382},
 'EVENTO': {'f1': 0.7692307692307693,
  'number': 8,
  'precision': 1.0,
  'recall': 0.625},
 'FUNDAMENTO': {'f1': 0.795505617977528,
  'number': 216,
  'precision': 0.7729257641921398,
  'recall': 0.8194444444444444},
 'LOCAL': {'f1': 0.8232931726907632,
  'number': 489,
  'precision': 0.8086785009861933,
  'recall': 0.8384458077709611},
 'ORGANIZACAO': {'f1': 0.8175824175824175,
  'number': 236,
  'precision': 0.8493150684931506,
  'recall': 0.788135593220339},
 'PESSOA': {'f1': 0.9136842105263158,
  'number': 476,
  'precision': 0.9156118143459916,
  'recall': 0.9117647058823529},
 'PRODUTODELEI': {'f1': 0.5180722891566265,
  'number': 158,
  'precision': 0.4942528735632184,
  'recall': 0.5443037974683544},
 'overall_accuracy': 0.971265041870939,
 'overall_f1': 0.8222285228239298,
 'overall_precision': 0.8114157806379407,
 'overall_recall': 0.8333333333333334}